<h1> Bangkok Housing Analysis</h1>

This project will analyse the districts and neighbourhood of Bangkok in term of the available facility in order to help people to decide the the suitable area to live in. This will be based on transport, restaurant, supermarket and other point of interest.

<h2>Import Relevant Libraries</h2>

<h3> Install Libraries </h3>

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} numpy pandas  matplotlib 

!conda install -c conda-forge geopy beautifulsoup4 scikit-learn folium=0.5.0 --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - matplotlib
    - numpy
    - pandas


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - beautifulsoup4
    - folium=0.5.0
    - geopy
    - scikit-learn


The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


<h3> Import Libraries </h3>

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<h2> Gather Bangkok district data </h2>

<h3>Import data from wikipedia using Beautiful Soup</h3>

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

print(soup.prettify()) 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of districts of Bangkok - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"efd1df51-36e2-485e-8fcd-35a20b16f937","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_districts_of_Bangkok","wgTitle":"List of districts of Bangkok","wgCurRevisionId":961269901,"wgRevisionId":961269901,"wgArticleId":29995815,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles containing Thai-language text","Districts of Bangkok","Thail

<h3>Collect the data from import page</h3>

In [4]:
postalCodes = [];
districts= [];
subdistricts = [];
latitude = [];
longitude = [];
population=[];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if columnNum == 1:
            districts.append(cell.string);   
            columnNum = 2
        elif columnNum == 2 :    
            columnNum = 3
        elif columnNum == 3 :
            postalCodes.append(cell.string);      
            columnNum = 4
        elif columnNum == 4 : 
            columnNum = 5
        elif columnNum == 5 : 
            population.append(cell.string); 
            columnNum = 6
        elif columnNum == 6 : 
            subdistricts.append(cell.string); 
            columnNum = 7
        elif columnNum == 7 : 
            latitude.append(cell.string); 
            columnNum = 8
        elif columnNum == 8 : 
            longitude.append(cell.string); 
            columnNum = 1
                
print('Data Collected.')

Data Collected.


<h3>Test Data Collection </h3>

In [5]:
print(postalCodes)

['10150', '10240', '10160', '10220', '10120', '10150', '10260', '10700', '10500', '10800', '10700', '10600', '10240', '10900\n', '10150\n', '10400\n', '10210\n', '10300\n', '10310\n', '10230\n', '10510\n', '10600\n', '10110\n', '10210\n', '10520\n', '10230\n', '10510\n', '10530\n', '10160\n', '10330\n', '10160\n', '10400\n', '10260\n', '10200\n', '10100\n', '10250\n', '10140\n', '10400\n', '10220\n', '10100\n', '10240\n', '10120\n', '10250\n', '10170\n', '10170\n', '10600\n', '10140\n', '10310\n', '10110\n', '10120\n', None]


<h3>Create Initial dataframe</h3>

In [229]:
column_names = ['PostalCode', 'Neighborhood', 'Sub-district','latitude','longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Neighborhood,Sub-district,latitude,longitude


<h3>Put collected data to the Dataframe</h3>

In [230]:
for data in range(len(districts)):
    if "\n" in postalCodes[data] :
        code = postalCodes[data][:-1]
    else :
        code = postalCodes[data]
    district = districts[data]
    sub_districts_num = subdistricts[data]
    lat = latitude[data]
    lon = longitude[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Neighborhood': district,
                                   'Sub-district': sub_districts_num[:-1],
                                    'latitude': lat[:-1],
                                    'longitude': lon[:-1]}, ignore_index=True)

TypeError: argument of type 'NoneType' is not iterable

In [231]:
neighbors

,PostalCode,Neighborhood,Sub-district,latitude,longitude
0,10150,Bang Bon,4,13.6592,100.3991
1,10240,Bang Kapi,2,13.765833,100.647778
2,10160,Bang Khae,4,13.696111,100.409444
3,10220,Bang Khen,2,13.873889,100.596389
4,10120,Bang Kho Laem,3,13.693333,100.5025
5,10150,Bang Khun Thian,2,13.660833,100.435833
6,10260,Bang Na,2,13.680081,100.5918
7,10700,Bang Phlat,4,13.793889,100.505
8,10500,Bang Rak,5,13.730833,100.524167
9,10800,Bang Sue,2,13.809722,100.537222


Bangkok have 50 district in total 

In [9]:
neighbors.shape

(50, 5)

<h2>Gathering relevant location Data from foursquare API</h2>

<h3>Get initial district location data</h3>

In [10]:
district_name = neighbors.loc[49, 'Neighborhood'] # neighborhood name
lat = neighbors.loc[49, 'latitude'] # neighborhood latitude value
lng = neighbors.loc[49, 'longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               lat, 
                                                               lng))


Latitude and longitude values of Yan Nawa are 13.696944, 100.543056.


<h3>Get initial nearby venue</h3>

In [91]:
CLIENT_ID='X42LYOEZGJAW2WD35SA4TB2LESK4BOMBKXPBM4QLGQY40NDZ'
CLIENT_SECRET='FTRC4MGDBI4LDG5VBA0NMAU1DXS14SPIC1UTP3ZILTAJEMTP'
radius = 1500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
VERSION = '20180323'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee4b0063639536919360573'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Yan Nawa',
  'headerFullLocation': 'Yan Nawa, Bangkok',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 13.710444013500014,
    'lng': 100.5569252238732},
   'sw': {'lat': 13.683443986499986, 'lng': 100.52918677612682}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b62d9a3f964a52074552ae3',
       'name': 'Tawandang German Brewery (โรงเบียร์เยอรมันตะวันแดง)',
       'contact': {},
       'location': {'address': '462/61 Rama III Rd.',
        'crossStreet': 'Naradhiwas Rajanagarindra Rd.',
        'lat': 13.6967

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Tawandang German Brewery (โรงเบียร์เยอรมันตะวั...,Brewery,13.696742,100.545958
1,Au Bon Pain (โอ บอง แปง),Café,13.697340,100.541133
2,Recreational Bangkok Biking,Bike Rental / Bike Share,13.698552,100.544010
3,Hachiban Ramen (ฮะจิบัง ราเมน),Ramen Restaurant,13.697124,100.540969
4,Est.33 by SINGHA,Brewery,13.700792,100.540887
...,...,...,...,...
70,Asavasopon Bose Rama3,Music Store,13.688191,100.539024
71,ร้านข้าว,Thai Restaurant,13.687050,100.545831
72,Cafe in a Bookyard,Coffee Shop,13.684803,100.544200
73,hayday,Beer Garden,13.685013,100.537903


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


<h3>Get nearby restaurant in each district </h3>

In [15]:
def getNearbyVenues(names, latitudes, longitudes,search_query):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            search_query,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Bangkok_venues = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                search_query = 'restaurant')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [20]:
print(Bangkok_venues.shape)
Bangkok_venues

(3831, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,Irashaimase Japanese Restaurant,13.658358,100.401403,Japanese Restaurant
2,Bang Bon,13.6592,100.3991,เจ๊ไน้ ก๋วยเตี๋ยวเป็ดตุ๋น,13.654137,100.405323,Noodle House
3,Bang Bon,13.6592,100.3991,หมูสะเต๊ะแม่กำไร,13.653925,100.391426,Satay Restaurant
4,Bang Bon,13.6592,100.3991,ส.สุขโภชนา,13.660358,100.404866,Chinese Restaurant
...,...,...,...,...,...,...,...
3826,Yan Nawa,13.696944,100.543056,หมูก๋วยเตี๋ยวแปดริ้ว,13.686867,100.536573,Noodle House
3827,Yan Nawa,13.696944,100.543056,Ciao Italian Restaurant,13.684772,100.544271,Italian Restaurant
3828,Yan Nawa,13.696944,100.543056,Little Pig หมูน้อย,13.684984,100.537980,Thai Restaurant
3829,Yan Nawa,13.696944,100.543056,ตุ๋ย ข้าวต้มบาทเดียว,13.684970,100.538017,Food


In [21]:
print('There are {} uniques categories.'.format(len(Bangkok_venues['Venue Category'].unique())))

There are 86 uniques categories.


In [22]:
Bangkok_res  = Bangkok_venues
Bangkok_res.loc[:,'Venue Category'] = 'Restaurant'
Bangkok_res

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Restaurant
1,Bang Bon,13.6592,100.3991,Irashaimase Japanese Restaurant,13.658358,100.401403,Restaurant
2,Bang Bon,13.6592,100.3991,เจ๊ไน้ ก๋วยเตี๋ยวเป็ดตุ๋น,13.654137,100.405323,Restaurant
3,Bang Bon,13.6592,100.3991,หมูสะเต๊ะแม่กำไร,13.653925,100.391426,Restaurant
4,Bang Bon,13.6592,100.3991,ส.สุขโภชนา,13.660358,100.404866,Restaurant
...,...,...,...,...,...,...,...
3826,Yan Nawa,13.696944,100.543056,หมูก๋วยเตี๋ยวแปดริ้ว,13.686867,100.536573,Restaurant
3827,Yan Nawa,13.696944,100.543056,Ciao Italian Restaurant,13.684772,100.544271,Restaurant
3828,Yan Nawa,13.696944,100.543056,Little Pig หมูน้อย,13.684984,100.537980,Restaurant
3829,Yan Nawa,13.696944,100.543056,ตุ๋ย ข้าวต้มบาทเดียว,13.684970,100.538017,Restaurant


<h3> Get nearby School in each district </h3>

In [25]:
Bangkok_school = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'school')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [26]:
print(Bangkok_school.shape)
Bangkok_school

(1181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Kapi,13.765833,100.647778,Net Design,13.767537,100.644157,School
1,Bang Kapi,13.765833,100.647778,NetDesign Bangkapi,13.767814,100.643950,School
2,Bang Kapi,13.765833,100.647778,Sakura Japanese School,13.766799,100.643451,School
3,Bang Kapi,13.765833,100.647778,CYES โรงเรียนชมอักษร,13.765311,100.642483,School
4,Bang Kapi,13.765833,100.647778,ECC (Thailand),13.766581,100.642461,School
...,...,...,...,...,...,...,...
1176,Yan Nawa,13.696944,100.543056,Barracuda School,13.694963,100.541523,Swim School
1177,Yan Nawa,13.696944,100.543056,สยามกลการ พระรามสาม,13.698452,100.537920,Music School
1178,Yan Nawa,13.696944,100.543056,Yamaha Music School,13.698398,100.537840,Music School
1179,Yan Nawa,13.696944,100.543056,Samit Music School,13.703093,100.535696,Music School


In [27]:
Bangkok_school_f = Bangkok_school[Bangkok_school['Venue Category']=="School"].reset_index(drop='true')
print(Bangkok_school_f.shape)
Bangkok_school_f

(509, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Kapi,13.765833,100.647778,Net Design,13.767537,100.644157,School
1,Bang Kapi,13.765833,100.647778,NetDesign Bangkapi,13.767814,100.643950,School
2,Bang Kapi,13.765833,100.647778,Sakura Japanese School,13.766799,100.643451,School
3,Bang Kapi,13.765833,100.647778,CYES โรงเรียนชมอักษร,13.765311,100.642483,School
4,Bang Kapi,13.765833,100.647778,ECC (Thailand),13.766581,100.642461,School
...,...,...,...,...,...,...,...
504,Yan Nawa,13.696944,100.543056,U.N.N. Academy,13.697486,100.532047,School
505,Yan Nawa,13.696944,100.543056,สารสาสน์เอกตรา,13.697954,100.531762,School
506,Yan Nawa,13.696944,100.543056,โรงเรียนสารสาสน์พิทยา,13.697230,100.530968,School
507,Yan Nawa,13.696944,100.543056,Sarasas Pittaya School,13.696586,100.530754,School


<h3> Get nearby public transport </h3>

<h4> Get Bts sky train station </h4>

In [28]:
Bangkok_bts = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'bts')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [44]:
print(Bangkok_bts.shape)
Bangkok_bts_f = Bangkok_bts[Bangkok_bts['Venue Category']=="Train Station"].reset_index(drop=True)
Bangkok_bts_f

(392, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Khen,13.873889,100.596389,BTS Wat Phra Si Maha That (N17) (BTS วัดพระศรี...,13.874526,100.596680,Train Station
1,Bang Rak,13.730833,100.524167,BTS Chong Nonsi (S3) (BTS ช่องนนทรี),13.723506,100.529415,Train Station
2,Bang Rak,13.730833,100.524167,BTS Surasak (S5) (BTS สุรศักดิ์),13.719249,100.521709,Train Station
3,Bang Rak,13.730833,100.524167,BTS Sala Daeng (S2) (BTS ศาลาแดง),13.728677,100.534689,Train Station
4,Bang Rak,13.730833,100.524167,[Construction Site] BTS ศึกษาวิทยา (Sueksa Wit...,13.721021,100.526404,Train Station
5,Bangkok Yai,13.722778,100.476389,BTS Talat Phlu (S10) (BTS ตลาดพลู),13.714288,100.476944,Train Station
6,Bangkok Yai,13.722778,100.476389,BTS Pho Nimit (S9) (BTS โพธิ์นิมิตร),13.719225,100.485756,Train Station
7,Bangkok Yai,13.722778,100.476389,BTS Wutthakat (S11) (BTS วุฒากาศ),13.712966,100.468820,Train Station
8,Chatuchak,13.828611,100.559722,BTS Ha Yaek Lat Phrao (N9) (BTS ห้าแยกลาดพร้าว),13.816640,100.562040,Train Station
9,Chatuchak,13.828611,100.559722,BTS Phahon Yothin 24 (N10) (BTS พหลโยธิน 24),13.822974,100.565865,Train Station


Get district underground train

In [30]:
Bangkok_mrt = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'mrt')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [31]:
print(Bangkok_mrt.shape)
Bangkok_mrt_f = Bangkok_mrt[Bangkok_mrt['Venue Category']=="Train Station"].reset_index(drop=True)
Bangkok_mrt_f

(116, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Phlat,13.793889,100.505,MRT Bang Phlat (BL07) (MRT บางพลัด),13.792171,100.504823,Train Station
1,Bang Phlat,13.793889,100.505,MRT Bang O (BL08) (MRT บางอ้อ),13.798962,100.509802,Train Station
2,Bang Rak,13.730833,100.524167,BTS Sala Daeng (S2) (BTS ศาลาแดง),13.728677,100.534689,Train Station
3,Bang Rak,13.730833,100.524167,BTS Chong Nonsi (S3) (BTS ช่องนนทรี),13.723506,100.529415,Train Station
4,Bang Rak,13.730833,100.524167,Bangkok Railway Station (SRT1001) (สถานีรถไฟกร...,13.739043,100.516712,Train Station
5,Bang Sue,13.809722,100.537222,MRT Tao Poon (PP16/BL10) (MRT เตาปูน),13.806117,100.530846,Train Station
6,Bang Sue,13.809722,100.537222,MRT Bang Son (PP15) (MRT บางซ่อน),13.820025,100.532456,Train Station
7,Bangkok Noi,13.770867,100.467933,MRT Bang Khun Non (BL04/OR01) (MRT บางขุนนนท์),13.763792,100.473694,Train Station
8,Bangkok Yai,13.722778,100.476389,MRT Bang Phai (BL33) (MRT บางไผ่),13.724560,100.465175,Train Station
9,Khlong San,13.730278,100.509722,Bangkok Railway Station (SRT1001) (สถานีรถไฟกร...,13.739043,100.516712,Train Station


<h4>Get premium public bus station</h4>

In [32]:
Bangkok_brt = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'brt')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [33]:
print(Bangkok_brt.shape)
Bangkok_brt_f = Bangkok_brt[Bangkok_brt['Venue Category']=="Bus Station"].reset_index(drop=True)
Bangkok_brt_f

(33, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Kho Laem,13.693333,100.5025,สถานีเจริญราษฎร์,13.692290,100.510560,Bus Station
1,Bang Kho Laem,13.693333,100.5025,BRT สะพานพระราม 3 (Rama III Bridge),13.693799,100.500349,Bus Station
2,Bang Kho Laem,13.693333,100.5025,BRT เจริญราษฎร์ (Charoen Rath),13.690147,100.504518,Bus Station
3,Bang Rak,13.730833,100.524167,BRT สาทร (Sathorn),13.721242,100.530825,Bus Station
4,Bangkok Yai,13.722778,100.476389,รถตู้ เดอะมอลล์ท่า่พระ - เซ็นทรัลพระราม 3,13.717436,100.478025,Bus Station
5,Bangkok Yai,13.722778,100.476389,BRT ราชพฤกษ์ (Ratchapruek),13.715685,100.479094,Bus Station
6,Rat Burana,13.682222,100.505556,สถานีเจริญราษฎร์,13.692290,100.510560,Bus Station
7,Rat Burana,13.682222,100.505556,BRT เจริญราษฎร์ (Charoen Rath),13.690147,100.504518,Bus Station
8,Rat Burana,13.682222,100.505556,BRT สะพานพระราม 9 (Rama IX Bridge),13.688114,100.515743,Bus Station
9,Rat Burana,13.682222,100.505556,BRT สะพานพระราม 3 (Rama III Bridge),13.693799,100.500349,Bus Station


<h4>Get district Pier</h4>

In [42]:
Bangkok_pier = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'pier')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [46]:
print(Bangkok_pier.shape)
Bangkok_pier

(178, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Kapi,13.765833,100.647778,ท่าเรือบางกะปิ (Bangkapi Pier) E21,13.765043,100.647365,Pier
1,Bang Kapi,13.765833,100.647778,ท่าเรือวัดศรีบุญเรือง (Wat Sriboonreung Pier) E22,13.767430,100.651550,Pier
2,Bang Kapi,13.765833,100.647778,ท่าเรือเดอะมอลล์บางกะปิ (The Mall Bangkapi Pie...,13.764608,100.643544,Pier
3,Bang Khae,13.696111,100.409444,Kanchanaphisek Bridge Pier (P13) (ท่าเรือสะพาน...,13.700279,100.406303,Pier
4,Bang Kho Laem,13.693333,100.5025,ริมแม่นำ้เจ้าพระยา | ท่าวัดอินทร์,13.688387,100.500423,Pier
...,...,...,...,...,...,...,...
173,Watthana,13.742222,100.585833,ท่าเรือวัดใหม่ช่องลม (Wat Mai Chong Lom Pier) E8,13.745803,100.574425,Pier
174,Watthana,13.742222,100.585833,ท่าเรือสะพานคลองตัน (Sapan Klongtun Pier) E13,13.741942,100.598291,Pier
175,Watthana,13.742222,100.585833,ท่าเรือรามหนึ่ง (Ramnueng Pier) E13-1,13.743337,100.598869,Pier
176,Watthana,13.742222,100.585833,ท่าเรือปรีดี พนมยงค์ สุขุมวิท 71,13.738204,100.598683,Pier


<h4> Combine All tables </h4>

In [51]:
t_frame = [Bangkok_bts_f,Bangkok_mrt_f,Bangkok_brt_f,Bangkok_pier]
Bangkok_public_transport = pd.concat(t_frame, ignore_index=True )
Bangkok_public_transport=Bangkok_public_transport.drop_duplicates().reset_index(drop=True)
Bangkok_public_transport.loc[:,'Venue Category'] = 'Public Transport'
Bangkok_public_transport

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Khen,13.873889,100.596389,BTS Wat Phra Si Maha That (N17) (BTS วัดพระศรี...,13.874526,100.596680,Public Transport
1,Bang Rak,13.730833,100.524167,BTS Chong Nonsi (S3) (BTS ช่องนนทรี),13.723506,100.529415,Public Transport
2,Bang Rak,13.730833,100.524167,BTS Surasak (S5) (BTS สุรศักดิ์),13.719249,100.521709,Public Transport
3,Bang Rak,13.730833,100.524167,BTS Sala Daeng (S2) (BTS ศาลาแดง),13.728677,100.534689,Public Transport
4,Bang Rak,13.730833,100.524167,[Construction Site] BTS ศึกษาวิทยา (Sueksa Wit...,13.721021,100.526404,Public Transport
...,...,...,...,...,...,...,...
239,Watthana,13.742222,100.585833,ท่าเรือวัดใหม่ช่องลม (Wat Mai Chong Lom Pier) E8,13.745803,100.574425,Public Transport
240,Watthana,13.742222,100.585833,ท่าเรือสะพานคลองตัน (Sapan Klongtun Pier) E13,13.741942,100.598291,Public Transport
241,Watthana,13.742222,100.585833,ท่าเรือรามหนึ่ง (Ramnueng Pier) E13-1,13.743337,100.598869,Public Transport
242,Watthana,13.742222,100.585833,ท่าเรือปรีดี พนมยงค์ สุขุมวิท 71,13.738204,100.598683,Public Transport


<h3>Get supermarket venue in each district</h3>

In [52]:
Bangkok_mall = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'Shopping Mall')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [53]:
print(Bangkok_mall.shape)
Bangkok_mall = Bangkok_mall[Bangkok_mall['Venue Category']=="Shopping Mall"].reset_index(drop='true')
Bangkok_mall

(343, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,Tesco Lotus Express thanon ekkachai bangbon,13.657758,100.397366,Shopping Mall
1,Bang Bon,13.6592,100.3991,NATUR Sale,13.656642,100.394428,Shopping Mall
2,Bang Bon,13.6592,100.3991,คาร์ฟู,13.661959,100.407408,Shopping Mall
3,Bang Bon,13.6592,100.3991,"ท๊อปซุปเปอร์มาร์เก็ต กรมอู่ทหารเรือ,,",13.655493,100.409574,Shopping Mall
4,Bang Kapi,13.765833,100.647778,The Mall Bangkapi (เดอะมอลล์ บางกะปิ),13.765816,100.642595,Shopping Mall
...,...,...,...,...,...,...,...
317,Yan Nawa,13.696944,100.543056,Hunan Lamei (หูหนานล่าเม่),13.700283,100.540212,Shopping Mall
318,Yan Nawa,13.696944,100.543056,NARA Place (นาราเพลส),13.700089,100.534806,Shopping Mall
319,Yan Nawa,13.696944,100.543056,Le Raffles,13.699980,100.534312,Shopping Mall
320,Yan Nawa,13.696944,100.543056,24 Mini Mart @ Le Rich สาธุประดิษฐ์,13.690341,100.534133,Shopping Mall


<h3>Get nearby hospital</h3>

In [59]:
Bangkok_hospital = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'Hospital')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [63]:
print(Bangkok_hospital.shape)
Bangkok_hospital_f = Bangkok_hospital[Bangkok_hospital['Venue Category']=="Hospital"].reset_index(drop='true')
Bangkok_hospital_f

(693, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,โรงพยาบาลจุฬา ตึก ภปร.ชั้น6,13.660062,100.402185,Hospital
1,Bang Bon,13.6592,100.3991,รพ. บางไผ่ บางแค,13.662179,100.403584,Hospital
2,Bang Bon,13.6592,100.3991,คลีนิกใจดีรักษาสัตว์,13.661813,100.404190,Hospital
3,Bang Bon,13.6592,100.3991,Bangpakok 8 Hospital (โรงพยาบาลบางปะกอก 8),13.662666,100.407112,Hospital
4,Bang Bon,13.6592,100.3991,โรงพยาบาลสัตว์พระราม2,13.658547,100.410200,Hospital
...,...,...,...,...,...,...,...
650,Yan Nawa,13.696944,100.543056,Mingkwan Clinic,13.698849,100.531543,Hospital
651,Yan Nawa,13.696944,100.543056,รพ.บ้านแพ้ว สาทร,13.707668,100.538250,Hospital
652,Yan Nawa,13.696944,100.543056,โรงพยาบาลบ้านแพ้ว สาขาสาทร ชั้น 10 TPI Tower,13.708037,100.538304,Hospital
653,Yan Nawa,13.696944,100.543056,BNH Pediatric Department,13.708276,100.537118,Hospital


<h3>Get nearby park</h3>

In [95]:
Bangkok_park = getNearbyVenues(names=districts,
                                 latitudes=neighbors['latitude'],
                                 longitudes=neighbors['longitude'],
                                 search_query = 'Parks')

Bang Bon
Bang Kapi
Bang Khae
Bang Khen
Bang Kho Laem
Bang Khun Thian
Bang Na
Bang Phlat
Bang Rak
Bang Sue
Bangkok Noi
Bangkok Yai
Bueng Kum
Chatuchak
Chom Thong
Din Daeng
Don Mueang
Dusit
Huai Khwang
Khan Na Yao
Khlong Sam Wa
Khlong San
Khlong Toei
Lak Si
Lat Krabang
Lat Phrao
Min Buri
Nong Chok
Nong Khaem
Pathum Wan
Phasi Charoen
Phaya Thai
Phra Khanong
Phra Nakhon
Pom Prap Sattru Phai
Prawet
Rat Burana
Ratchathewi
Sai Mai
Samphanthawong
Saphan Sung
Sathon
Suan Luang
Taling Chan
Thawi Watthana
Thon Buri
Thung Khru
Wang Thonglang
Watthana
Yan Nawa


In [97]:
print(Bangkok_park.shape)
Bangkok_park_f = Bangkok_park[Bangkok_park['Venue Category']=="Park"].reset_index(drop='true')
Bangkok_park_f

(232, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,Prinsiri Park&Clubhouse,13.668169,100.406974,Park
1,Bang Kapi,13.765833,100.647778,Suan Son (สวนสน),13.762940,100.648895,Park
2,Bang Kapi,13.765833,100.647778,สวนเฉลิมพระเกียรติ 72 พรรษา,13.756339,100.651186,Park
3,Bang Kapi,13.765833,100.647778,"สวนน้ำพัฒนธารา , นิด้า",13.771211,100.655258,Park
4,Bang Khae,13.696111,100.409444,Kanlapaphruek Park (Suan Liap) (สวนกัลปพฤกษ์ (...,13.683297,100.407517,Park
...,...,...,...,...,...,...,...
208,Watthana,13.742222,100.585833,Natural Park Botanic Gardens,13.733279,100.576977,Park
209,Watthana,13.742222,100.585833,ทางเลี้ยวใกล้ทางด่วนรามอินทรา,13.752313,100.590999,Park
210,Yan Nawa,13.696944,100.543056,Park@ILC,13.691267,100.535563,Park
211,Yan Nawa,13.696944,100.543056,Pech's Balcony Garden,13.708801,100.538404,Park


<h3>Combine All DataFrame</h3>

In [122]:
all_frame = [Bangkok_public_transport,Bangkok_mall,Bangkok_school_f,Bangkok_hospital_f,Bangkok_park_f]
Bangkok_all_venue = pd.concat(all_frame, ignore_index=True).sort_values(by=['Neighborhood','Venue Category']).reset_index(drop='true')
Bangkok_all_venue

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,โรงพยาบาลจุฬา ตึก ภปร.ชั้น6,13.660062,100.402185,Hospital
1,Bang Bon,13.6592,100.3991,รพ. บางไผ่ บางแค,13.662179,100.403584,Hospital
2,Bang Bon,13.6592,100.3991,คลีนิกใจดีรักษาสัตว์,13.661813,100.404190,Hospital
3,Bang Bon,13.6592,100.3991,Bangpakok 8 Hospital (โรงพยาบาลบางปะกอก 8),13.662666,100.407112,Hospital
4,Bang Bon,13.6592,100.3991,โรงพยาบาลสัตว์พระราม2,13.658547,100.410200,Hospital
...,...,...,...,...,...,...,...
1938,Yan Nawa,13.696944,100.543056,Hunan Lamei (หูหนานล่าเม่),13.700283,100.540212,Shopping Mall
1939,Yan Nawa,13.696944,100.543056,NARA Place (นาราเพลส),13.700089,100.534806,Shopping Mall
1940,Yan Nawa,13.696944,100.543056,Le Raffles,13.699980,100.534312,Shopping Mall
1941,Yan Nawa,13.696944,100.543056,24 Mini Mart @ Le Rich สาธุประดิษฐ์,13.690341,100.534133,Shopping Mall


<h2>Data Preprocessing</h2>

<h3>Covert data to onehot state</h3>

In [250]:
# one hot encoding
bangkok_onehot = pd.get_dummies(Bangkok_all_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = Bangkok_all_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangkok_onehot.columns[-1]] + list(bangkok_onehot.columns[:-1])
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot

,Neighborhood,Hospital,Park,Public Transport,School,Shopping Mall
0,Bang Bon,1,0,0,0,0
1,Bang Bon,1,0,0,0,0
2,Bang Bon,1,0,0,0,0
3,Bang Bon,1,0,0,0,0
4,Bang Bon,1,0,0,0,0
...,...,...,...,...,...,...
1938,Yan Nawa,0,0,0,0,1
1939,Yan Nawa,0,0,0,0,1
1940,Yan Nawa,0,0,0,0,1
1941,Yan Nawa,0,0,0,0,1


In [251]:
bangkok_onehot.shape

(1943, 6)

<h3>Group the data</h3>

In [285]:
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').agg({'Hospital': 'sum', 'Park': 'sum','Public Transport': 'sum','School': 'sum','Shopping Mall': 'sum'}).reset_index()
bangkok_grouped

,Neighborhood,Hospital,Park,Public Transport,School,Shopping Mall
0,Bang Bon,6,1,0,0,4
1,Bang Kapi,38,3,3,13,14
2,Bang Khae,3,2,1,1,3
3,Bang Khen,5,6,1,6,2
4,Bang Kho Laem,7,8,10,2,12
5,Bang Khun Thian,7,3,0,8,6
6,Bang Na,4,2,5,2,1
7,Bang Phlat,24,4,11,17,5
8,Bang Rak,53,10,22,21,21
9,Bang Sue,4,1,2,3,1


In [286]:
bangkok_grouped.shape

(50, 6)

<h2>Modelling Data</h2>

In [287]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

bangkok_grouped_clustering = bangkok_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 2, 1, 0])

In [288]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
bangkok_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

bangkok_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangkok_merged = bangkok_merged.join(bangkok_grouped.set_index('Neighborhood'), on='Neighborhood')

bangkok_merged # check the last columns!

,PostalCode,Neighborhood,Sub-district,latitude,longitude,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
0,10150,Bang Bon,4,13.6592,100.3991,0,6,1,0,0,4
1,10240,Bang Kapi,2,13.765833,100.647778,2,38,3,3,13,14
2,10160,Bang Khae,4,13.696111,100.409444,0,3,2,1,1,3
3,10220,Bang Khen,2,13.873889,100.596389,0,5,6,1,6,2
4,10120,Bang Kho Laem,3,13.693333,100.5025,0,7,8,10,2,12
5,10150,Bang Khun Thian,2,13.660833,100.435833,0,7,3,0,8,6
6,10260,Bang Na,2,13.680081,100.5918,0,4,2,5,2,1
7,10700,Bang Phlat,4,13.793889,100.505,2,24,4,11,17,5
8,10500,Bang Rak,5,13.730833,100.524167,1,53,10,22,21,21
9,10800,Bang Sue,2,13.809722,100.537222,0,4,1,2,3,1


<h2>Visualise Result </h2>

In [296]:

# create map
map_clusters = folium.Map(location=[13.77, 100.6], zoom_start=11.45)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(bangkok_merged['latitude'], bangkok_merged['longitude'], bangkok_merged['Neighborhood'], bangkok_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        location=[float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [290]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 0, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,Neighborhood,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
0,Bang Bon,0,6,1,0,0,4
2,Bang Khae,0,3,2,1,1,3
3,Bang Khen,0,5,6,1,6,2
4,Bang Kho Laem,0,7,8,10,2,12
5,Bang Khun Thian,0,7,3,0,8,6
6,Bang Na,0,4,2,5,2,1
9,Bang Sue,0,4,1,2,3,1
11,Bangkok Yai,0,9,5,9,8,8
12,Bueng Kum,0,4,3,0,7,4
13,Chatuchak,0,5,7,3,11,6


In [291]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 1, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,Neighborhood,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
8,Bang Rak,1,53,10,22,21,21
37,Ratchathewi,1,75,8,11,36,36
48,Watthana,1,69,5,9,16,23


In [292]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 2, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,Neighborhood,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
1,Bang Kapi,2,38,3,3,13,14
7,Bang Phlat,2,24,4,11,17,5
15,Din Daeng,2,22,6,2,17,2
17,Dusit,2,50,2,0,22,3
31,Phaya Thai,2,27,8,1,14,6
41,Sathon,2,27,10,5,31,6


In [293]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 3, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,Neighborhood,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
21,Khlong San,3,20,5,45,20,17
39,Samphanthawong,3,20,5,42,20,15


In [294]:
bangkok_merged.loc[bangkok_merged['Cluster Labels'] == 4, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,Neighborhood,Cluster Labels,Hospital,Park,Public Transport,School,Shopping Mall
10,Bangkok Noi,4,11,6,2,17,5
29,Pathum Wan,4,18,13,8,32,30
33,Phra Nakhon,4,7,8,18,25,3
34,Pom Prap Sattru Phai,4,13,3,2,22,5
45,Thon Buri,4,4,4,5,21,5
49,Yan Nawa,4,5,3,3,15,8
